In [29]:
import py3Dmol
import copy
import itertools
from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import rdBase
from rdkit.Chem import rdMolAlign
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
import numpy as np
import itertools
import os
p = AllChem.ETKDGv2()
p.randomSeed = 42
p.verbose = False

#based on 
https://iwatobipen.wordpress.com/2018/09/26/3d-alignment-function-of-rdkit-rdkit/ 
https://www.rdkit.org/UGM/2012/Ebejer_20110926_RDKit_1stUGM.pdf


In [21]:
def generate_conformers(mol, num = 50, addHs = False, stereo = False, draw = False):
    """Generate stereoisomers and conformers, returns list object
    mol: is rdkit molecule object
    num: number of conformers to generate (per isomer)
    add_hs: whether to add hydrogens or not
    stereo: whether or not to generate stereoisomers
    draw: whether to draw the confermers"""

    if addHs:
        mol = Chem.AddHs(mol)


    if 7 < Chem.rdMolDescriptors.CalcNumRotatableBonds(mol) <= 12 and num == 50:
        print('The number of rotatable bonds is bigger than 7, num is increased to 200')
        num = 200
    elif Chem.rdMolDescriptors.CalcNumRotatableBonds(mol) > 12 and num == 50:
        print('The number of rotatable bonds is bigger than 12, num is increased to 300')
        num = 300

    # generate isomers
    conformers = []

    if stereo:
        stereo_isomers = tuple(EnumerateStereoisomers(mol))
        # generate conformers per each stereoisomer
        for isomer in stereo_isomers:
            AllChem.EmbedMultipleConfs(isomer, num, p)
            conformers.append(isomer)
    else:
        AllChem.EmbedMultipleConfs(mol, num, p)
        conformers.append(mol)

    if draw:
        Draw.MolsToGridImage(conformers)

    return conformers


In [22]:
def optimize_conformation(mol, addHs=True):
    """optimize conformation of molecule 
    mol: is rdkit molecule object
    addHs: whether to add hydrogens or not
    """

    if addHs:
        mol = Chem.AddHs(mol)
        
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol) 

    return mol

In [23]:
def align_conformers(mol, ref):
    """Align and score conformers of mol to reference, returns best pair
    ref: is rdkit molecule of template / reference
    mol: is rdkit molecule object"""

    # conduct GetCrippenO3A
    # Get an O3A object with atomMap and weights vectors to overlay
    # the probe molecule onto the reference molecule based on Crippen logP atom contributions
    crippen_contribs = rdMolDescriptors._CalcCrippenContribs(mol)
    crippen_ref = rdMolDescriptors._CalcCrippenContribs(ref)

    tempscore = []
    ids = []
    for prb_cid in range(mol.GetNumConformers()):
        for ref_cid in range(ref.GetNumConformers()):
            crippenO3A = rdMolAlign.GetCrippenO3A(prbMol = mol, refMol = ref, prbCrippenContribs = crippen_contribs,
                                                refCrippenContribs = crippen_ref, prbCid = prb_cid, refCid = ref_cid)
            crippenO3A.Align()
            tempscore.append(crippenO3A.Score())
            ids.append([prb_cid, ref_cid])
    best = np.argmax(tempscore)
    # select best isomer
    best_mol = Chem.MolFromMolBlock(Chem.MolToMolBlock(mol, confId=int(ids[best][0])), removeHs=False)
    best_ref = Chem.MolFromMolBlock(Chem.MolToMolBlock(ref, confId=int(ids[best][1])), removeHs=False)

    return best_mol, best_ref

In [54]:
def align_mcs(mol,ref):

    res=rdFMCS.FindMCS([mol,ref],threshold=0.9, matchValences=True, ringMatchesRingOnly=True, completeRingsOnly=True)
    p = Chem.MolFromSmarts(res.smartsString)
    p
    
    matchingMols = [x for x in [mol,ref] if x.HasSubstructMatch(p)]
    print(matchingMols)
    core1 = AllChem.DeleteSubstructs(AllChem.ReplaceSidechains(Chem.RemoveHs(matchingMols[1]),p),Chem.MolFromSmiles('*'))
    core1.UpdatePropertyCache()

    return core1


In [49]:
def main(to_align, intermediate, ref):
    """Code for optimizing the conformation, generating conformers and aligning conformers to reference."""
    
    # get lowest energy conformation
    to_align = optimize_conformation(to_align, addHs=True)
    intermediate = optimize_conformation(intermediate, addHs=True)
    ref = optimize_conformation(ref, addHs=True)

    # generate isomers
    to_align = generate_conformers(to_align, num = 50, addHs = False, stereo = False, draw = False)[0]
    intermediate = generate_conformers(intermediate, num = 50, addHs = False, stereo = False, draw = False)[0]
    ref = generate_conformers(ref, num = 1, addHs = False, stereo = False, draw = False)[0]

    # select best conformer based on alignment score
    best_mol, best_ref = align_conformers(to_align, ref)

    best_intermediate, _ = align_conformers(intermediate, ref)

    return best_mol, best_intermediate, best_ref

In [55]:
# load molecules
sdf = False
ligands  = [m for m in Chem.SDMolSupplier('lig.sdf')]
intermediates = [m for m in Chem.SDMolSupplier('interm.sdf')]
for i, intermediate in enumerate(intermediates):
    
    if i != 14: continue
    print(intermediate.GetProp("_Name"))
    os.makedirs(f'3d/{i}', exist_ok = True)
    # first entry in list of ligands is reference, second ligand from pair will be aligned to this
    ref = ligands.pop(13*2)
    to_align = ligands.pop(13*2)
    core = align_mcs(to_align, ref)
    p_crippen = py3Dmol.view(width=600, height=400)
    p_crippen.addModel(Chem.MolToMolBlock(core), 'sdf')
    

    p_crippen.setStyle({'stick':{}})
    p_crippen.render()
    best_mol, best_intermediate, best_ref = main(to_align, intermediate, ref)

    # write to file
    if sdf:
        with Chem.SDWriter('3d/1/conformers_test.sdf') as w:
            best_mol.SetProp('_Name', to_align.GetProp("_Name"))
            w.write(best_mol)

        with Chem.SDWriter('conformers_test.sdf') as w:
            best_mol.SetProp('_Name', intermediate.GetProp("_Name"))
            w.write(best_mol)    

        with Chem.SDWriter('ref_test.sdf') as w:
            best_ref.SetProp('_Name', ref.GetProp("_Name"))
            w.write(best_ref)

    with Chem.PDBWriter(f'3d/{i}/{to_align.GetProp("_Name")}.pdb') as w:
        w.write(best_mol)

    with Chem.PDBWriter(f'3d/{i}/{intermediate.GetProp("_Name")}.pdb') as w:
        w.write(best_intermediate)

    with Chem.PDBWriter(f'3d/{i}/{ref.GetProp("_Name")}.pdb') as w:
        w.write(best_ref)

cats_CatS_29_CatS_141_i
[<rdkit.Chem.rdchem.Mol object at 0x7fa5c539b880>, <rdkit.Chem.rdchem.Mol object at 0x7fa5c539b940>]


In [66]:

mh = Chem.AddHs(to_align)
AllChem.EmbedMolecule(mh,randomSeed=0xf00d)

m_match = mh.GetSubstructMatch(core)

ref.GetSubstructMatch(core)

newmh = Chem.AddHs(ref)
newm_match = newmh.GetSubstructMatch(core)
cmap = {newm_match[i]:mh.GetConformer().GetAtomPosition(m_match[i]) for i in range(len(m_match))}
AllChem.EmbedMolecule(newmh,randomSeed=0xf00d,coordMap=cmap)

AllChem.AlignMol(newmh,mh,atomMap = list(zip(newm_match,m_match)))


p_crippen = py3Dmol.view(width=600, height=400)
p_crippen.addModel(Chem.MolToMolBlock(newmh), 'sdf')
p_crippen.addModel(Chem.MolToMolBlock(mh), 'sdf')
p_crippen.setStyle({'stick':{}})
p_crippen.render()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [60]:
GetFF=lambda x,confId=-1:AllChem.MMFFGetMoleculeForceField(x,AllChem.MMFFGetMoleculeProperties(x),confId=confId)
p_crippen = py3Dmol.view(width=600, height=400)
p_crippen.addModel(Chem.MolToMolBlock(core), 'sdf')
for m in [to_align]:
    print(m.GetProp("_Name"))
    nm = Chem.AddHs(m)
    
    AllChem.ConstrainedEmbed(nm,core,getForceField=GetFF)
    p_crippen.addModel(Chem.MolToMolBlock(nm), 'sdf')
p_crippen.setStyle({'stick':{}})
p_crippen.render()

cats_CatS_132


RDKit WARNING: [14:23:02] Could not triangle bounds smooth molecule.
[14:23:02] Could not triangle bounds smooth molecule.


ValueError: Could not embed molecule.

In [ ]:
p_crippen = py3Dmol.view(width=600, height=400)
#p_crippen.addModel(Chem.MolToMolBlock(best_ref), 'sdf')

#p_crippen.addModel(Chem.MolToMolBlock(best_mol), 'sdf')
p_crippen.addModel(Chem.MolToMolBlock(best_intermediate), 'sdf')
p_crippen.setStyle({'stick':{}})
p_crippen.render()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol